- https://towardsdatascience.com/six-spark-exercises-to-rule-them-all-242445b24565
- https://intellipaat.com/community/6443/how-to-optimize-shuffle-spill-in-apache-spark-application#:~:text=Spark%201.4%20has%20some%20better,available%20for%20the%20shuffle%20buffer.&text=Increase%20the%20memory%20in%20your,executor.

export SPARK_HOME=/opt/spark

In [1]:
import os

In [2]:
os.environ['SPARK_HOME'] = '/opt/spark'

In [3]:
import findspark

In [4]:
findspark.init()

In [5]:
import pyspark

In [6]:
pyspark.__file__

'/opt/spark/python/pyspark/__init__.py'

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

c = F.col

In [4]:
spark = (
    SparkSession
    .builder
    .master('local[*]')
    .config(
        'spark.sql.shuffle.partitions',
        '400'
    )
    .config("spark.sql.autoBroadcastJoinThreshold", -1)
    .config('spark.driver.memory', '5g')
    .appName('hello-spark')
    .getOrCreate()
)

In [5]:
spark

## Warmup 1


    Find out how many orders, how many products and how many sellers are in the data.
    How many products have been sold at least once? Which is the product contained in more orders?

In [6]:
sdf_products = spark.read.parquet('/data/dataset/products_parquet')
sdf_products.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- price: string (nullable = true)



In [7]:
sdf_seller = spark.read.parquet('/data/dataset/sellers_parquet')
sdf_seller.printSchema()

root
 |-- seller_id: string (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- daily_target: string (nullable = true)



In [8]:
sdf_order = spark.read.parquet("/data/dataset/sales_parquet")
sdf_order.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- num_pieces_sold: string (nullable = true)
 |-- bill_raw_text: string (nullable = true)



## Warm Up 2

    How many distinct products have been sold in each day?

## Excercise 1

    What is the average revenue of the orders?

### salting

## Excercise 2

    For each seller, what is the average % contribution of an order to the seller's daily quota?

## Exercise 3

    Who are the second most selling and the least selling persons (sellers) for each product?
    Who are those for product with `product_id = 0`

- https://databricks.com/blog/2015/07/15/introducing-window-functions-in-spark-sql.html

In [15]:
from pyspark.sql.window import Window

```python
spec = Window.partitionBy(*cols).orderBy(*cols)
windFunc.over(spec)
...
```

## Excercise 4

    Create a new column called "hashed_bill" defined as follows:

        - if the order_id is even: apply MD5 hashing iteratively to the bill_raw_text field,
          once for each 'A' (capital 'A') present in the text. E.g. if the bill text is 'nbAAnllA',
          you would apply hashing three times iteratively (only if the order number is even)
        - if the order_id is odd: apply SHA256 hashing to the bill text

    Finally, check if there are any duplicate on the new column